In [41]:
import pandas as pd
import datetime
import pickle

from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping

### Load Scalers and Preprocessors

In [2]:
with open("artifacts/preprocessing_classifier/gender_encoder.pkl", 'rb') as file:
    gender_encoder = pickle.load(file)
with open("artifacts/preprocessing_classifier/geography_encoder.pkl", 'rb') as file:
    geography_encoder = pickle.load(file)
with open("artifacts/preprocessing_classifier/scaler.pkl", 'rb') as file:
    scaler = pickle.load(file)

### Load the Data

In [7]:
data = pd.read_csv("data/Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
data.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1, inplace = True)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
data['Gender'] = gender_encoder.transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [10]:
geo_encoded_data = pd.DataFrame(data = geography_encoder.transform(data[['Geography']]), columns = geography_encoder.get_feature_names_out())
data = pd.concat([data, geo_encoded_data], axis = 1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,France,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,France,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [11]:
data.drop("Geography", axis = 1, inplace = True)

In [12]:
X = data.drop("Exited", axis = 1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 67)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### Hyperparameter Tuning

In [49]:
def create_model(neurons = 64, hidden_layers = 1, shrink = 1):
    model = Sequential()
    model.add(Input((X_train.shape[1],)))

    for _ in range(hidden_layers):
        model.add(Dense(neurons, activation = 'relu'))
        neurons = neurons // shrink

    model.add(Dense(1, activation = 'sigmoid'))

    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01)
    loss = tf.keras.losses.BinaryCrossentropy()

    model.compile(
        optimizer = optimizer,
        loss = loss,
        metrics = ['accuracy']
    )

    return model

In [50]:
early_stopping = EarlyStopping(monitor = 'loss', patience = 5, restore_best_weights = True)

model = KerasClassifier(neurons = 64, 
                        hidden_layers = 1,
                        shrink = 1,
                        build_fn = create_model, 
                        epochs = 50, 
                        batch_size = 10, 
                        verbose = 0,
                        callbacks = [early_stopping])

In [51]:
params = {
    'neurons': [32, 64],
    'hidden_layers': [1, 2],
    'shrink': [1, 2],
    'epochs': [50, 100]
}

In [53]:
grid = GridSearchCV(estimator = model, cv = 2, n_jobs = 1, param_grid = params, verbose = 2)
grid_result = grid.fit(X_train, y_train)

print(f'Best: {grid_result.best_score_}\nBest Params: {grid_result.best_params_}')

Fitting 2 folds for each of 16 candidates, totalling 32 fits


c:\Users\User\Documents\Projects_Udemy\ANN_project\venv\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


[CV] END ...epochs=50, hidden_layers=1, neurons=32, shrink=1; total time=  16.4s


c:\Users\User\Documents\Projects_Udemy\ANN_project\venv\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


[CV] END ...epochs=50, hidden_layers=1, neurons=32, shrink=1; total time=  16.9s


...

[CV] END ..epochs=100, hidden_layers=2, neurons=64, shrink=2; total time=  22.4s


c:\Users\User\Documents\Projects_Udemy\ANN_project\venv\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


[CV] END ..epochs=100, hidden_layers=2, neurons=64, shrink=2; total time=  20.2s


c:\Users\User\Documents\Projects_Udemy\ANN_project\venv\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Best: 0.8555
Best Params: {'epochs': 50, 'hidden_layers': 1, 'neurons': 32, 'shrink': 2}
